# Author: Mrinmoy sarkar
# email: mrinmoy.pol@gmail.com
# date: 9/27/2018

In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style('white')
from sklearn.model_selection import train_test_split
import time  

from keras.preprocessing.image import load_img

import tensorflow as tf

# companion functions and classes

In [84]:
img_size_ori = 101
img_size_target = 128

def unroll(img):
    if img_size_ori == img_size_target:
        return img
    return img.ravel()

class batch_generator():
    def __init__(self,X,y):
        self.index = 0
        self.images = X['images'].apply(unroll).values
        self.masks = y['masks'].apply(unroll).values
        self.max_index = X.shape[0]
        
    def get_next_batch(self,batch_size):
        if self.index+batch_size <= self.max_index:
            x = self.images[self.index:self.index+batch_size]
            y = self.masks[self.index:self.index+batch_size]
            self.index += batch_size
        elif (self.max_index - self.index) > 0:
            x = self.images[self.index:self.max_index]
            y = self.masks[self.index:self.max_index]
            self.index = self.max_index
        else:
            self.reset_index()
            x = self.images[self.index:self.index+batch_size]
            y = self.masks[self.index:self.index+batch_size]
            self.index += batch_size
        return np.array(x.tolist()),np.array(y.tolist())
    
    def get_all_data(self):
        x = self.images[0:self.max_index]
        y = self.masks[0:self.max_index]
        return np.array(x.tolist()),np.array(y.tolist())
    
    def reset_index(self):
        self.index = 0

# Load data from file

In [85]:
train_df = pd.read_csv("../data/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../data/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

In [86]:
train_df["images"] = [np.array(load_img("../data/train/images/{}.png".format(idx), color_mode="grayscale")) / 255 for idx in (train_df.index)]
train_df["masks"] = [np.array(load_img("../data/train/masks/{}.png".format(idx), color_mode="grayscale")) / 255 for idx in (train_df.index)]

In [87]:
train_df["images"][0].shape

(101, 101)

# Split data into train and test set

In [88]:
X = pd.DataFrame(data=train_df["images"],columns=['images'])
y = pd.DataFrame(data=train_df["masks"],columns=['masks'])
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=101)
batch_gen = batch_generator(X_train,y_train)
testbatch_gen = batch_generator(X_test,y_test)
total_no_of_train_samples = X_train.shape[0]

In [91]:
learning_rate = 0.001

#with tf.device('/gpu:0'):
# if True:
x = tf.placeholder(tf.float32,shape=[None,img_size_ori*img_size_ori])
y_true = tf.placeholder(tf.float32,shape=[None,img_size_ori*img_size_ori])

inputs_ori = tf.reshape(x,[-1,img_size_ori,img_size_ori,1])#tf.placeholder(tf.float32,shape=(None,128,128,1),name='inputs')
targets_ori = tf.reshape(y_true,[-1,img_size_ori,img_size_ori,1])#tf.placeholder(tf.float32,shape=(None,128,128,1),name='targets')

inputs = tf.image.resize_images(inputs_ori,size=(128,128),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#     targets = tf.image.resize_images(targets_ori,size=(128,128),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)


conv1 = tf.layers.conv2d(inputs=inputs,filters=64,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
maxpool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=(2,2),strides=(2,2),padding='same')

conv2 = tf.layers.conv2d(inputs=maxpool1,filters=32,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
maxpool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=(2,2),strides=(2,2),padding='same')

conv3 = tf.layers.conv2d(inputs=maxpool2,filters=16,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
maxpool3 = tf.layers.max_pooling2d(inputs=conv3,pool_size=(2,2),strides=(2,2),padding='same')

conv4 = tf.layers.conv2d(inputs=maxpool3,filters=16,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
maxpool4 = tf.layers.max_pooling2d(inputs=conv4,pool_size=(2,2),strides=(2,2),padding='same')

conv5 = tf.layers.conv2d(inputs=maxpool4,filters=16,kernel_size=(3,3),padding='same',activation=tf.nn.relu)
encoded = tf.layers.max_pooling2d(inputs=conv5,pool_size=(2,2),strides=(2,2),padding='same')


upsampler1 = tf.image.resize_images(encoded,size=(8,8),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv6 = tf.layers.conv2d(inputs=upsampler1,filters=16,kernel_size=(3,3),padding='same',activation=tf.nn.relu)

upsampler2 = tf.image.resize_images(conv6,size=(16,16),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv7 = tf.layers.conv2d(inputs=upsampler2,filters=16,kernel_size=(3,3),padding='same',activation=tf.nn.relu)

upsampler3 = tf.image.resize_images(conv7,size=(32,32),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv8 = tf.layers.conv2d(inputs=upsampler3,filters=16,kernel_size=(3,3),padding='same',activation=tf.nn.relu)

upsampler4 = tf.image.resize_images(conv8,size=(64,64),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv9 = tf.layers.conv2d(inputs=upsampler4,filters=32,kernel_size=(3,3),padding='same',activation=tf.nn.relu)

upsampler5 = tf.image.resize_images(conv9,size=(128,128),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
conv10 = tf.layers.conv2d(inputs=upsampler5,filters=64,kernel_size=(3,3),padding='same',activation=tf.nn.relu)

downsampler = tf.image.resize_images(conv10,size=(img_size_ori,img_size_ori),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

logits = tf.layers.conv2d(inputs=downsampler,filters=1,kernel_size=(3,3),padding='same',activation=None)
decoded = tf.nn.sigmoid(logits)
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_ori,logits=logits)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(cost)



In [93]:
t =time.time()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    options = tf.RunOptions(output_partition_graphs=True)
    metadata = tf.RunMetadata()
    sess.run(tf.global_variables_initializer(),options=options, run_metadata=metadata)
    epochs = 5000
    batch_size = 100
    for e in range(epochs):
        for ii in range(total_no_of_train_samples//batch_size):
            x_train_batch,y_train_batch = batch_gen.get_next_batch(batch_size=batch_size)
            batch_cost, _ = sess.run([cost,optimizer],feed_dict={x:x_train_batch,y_true:y_train_batch})
#             print("Trainning loss: {:.4f}".format(batch_cost))
#             print(sess.run([cost,optimizer],feed_dict={x:x_train,y_true:y_train},options=options, run_metadata=metadata))
#             print(metadata.partition_graphs)
        print("epoch No.: ", e)
    training_time = time.time()-t
    print('training time: ', training_time)
    tr_loss = 0
    te_loss = 0
    n = 0
    t = time.time()
    for i in range(2):
        for ii in range(total_no_of_train_samples//batch_size):
            xt,yt = batch_gen.get_next_batch(batch_size=batch_size)
            training_loss = sess.run(cost,feed_dict={x:xt,y_true:yt})
            tr_loss += training_loss
            
            xtest,ytest = testbatch_gen.get_next_batch(batch_size=batch_size)
            test_loss = sess.run(cost,feed_dict={x:xtest,y_true:ytest})
            te_loss += test_loss
            n += 1 
    print('training loss: ', tr_loss/n, ' test loss: ', te_loss/n, 'test time: ', time.time()-t)


epoch No.:  0
epoch No.:  1
epoch No.:  2
epoch No.:  3
epoch No.:  4
epoch No.:  5
epoch No.:  6
epoch No.:  7
epoch No.:  8
epoch No.:  9
epoch No.:  10
epoch No.:  11
epoch No.:  12
epoch No.:  13
epoch No.:  14
epoch No.:  15
epoch No.:  16
epoch No.:  17
epoch No.:  18
epoch No.:  19
epoch No.:  20
epoch No.:  21
epoch No.:  22
epoch No.:  23
epoch No.:  24
epoch No.:  25
epoch No.:  26
epoch No.:  27
epoch No.:  28
epoch No.:  29
epoch No.:  30
epoch No.:  31
epoch No.:  32
epoch No.:  33
epoch No.:  34
epoch No.:  35
epoch No.:  36
epoch No.:  37
epoch No.:  38
epoch No.:  39
epoch No.:  40
epoch No.:  41
epoch No.:  42
epoch No.:  43
epoch No.:  44
epoch No.:  45
epoch No.:  46
epoch No.:  47
epoch No.:  48
epoch No.:  49
epoch No.:  50
epoch No.:  51
epoch No.:  52
epoch No.:  53
epoch No.:  54
epoch No.:  55
epoch No.:  56
epoch No.:  57
epoch No.:  58
epoch No.:  59
epoch No.:  60
epoch No.:  61
epoch No.:  62
epoch No.:  63
epoch No.:  64
epoch No.:  65
epoch No.:  66
epoch

epoch No.:  518
epoch No.:  519
epoch No.:  520
epoch No.:  521
epoch No.:  522
epoch No.:  523
epoch No.:  524
epoch No.:  525
epoch No.:  526
epoch No.:  527
epoch No.:  528
epoch No.:  529
epoch No.:  530
epoch No.:  531
epoch No.:  532
epoch No.:  533
epoch No.:  534
epoch No.:  535
epoch No.:  536
epoch No.:  537
epoch No.:  538
epoch No.:  539
epoch No.:  540
epoch No.:  541
epoch No.:  542
epoch No.:  543
epoch No.:  544
epoch No.:  545
epoch No.:  546
epoch No.:  547
epoch No.:  548
epoch No.:  549
epoch No.:  550
epoch No.:  551
epoch No.:  552
epoch No.:  553
epoch No.:  554
epoch No.:  555
epoch No.:  556
epoch No.:  557
epoch No.:  558
epoch No.:  559
epoch No.:  560
epoch No.:  561
epoch No.:  562
epoch No.:  563
epoch No.:  564
epoch No.:  565
epoch No.:  566
epoch No.:  567
epoch No.:  568
epoch No.:  569
epoch No.:  570
epoch No.:  571
epoch No.:  572
epoch No.:  573
epoch No.:  574
epoch No.:  575
epoch No.:  576
epoch No.:  577
epoch No.:  578
epoch No.:  579
epoch No

epoch No.:  1028
epoch No.:  1029
epoch No.:  1030
epoch No.:  1031
epoch No.:  1032
epoch No.:  1033
epoch No.:  1034
epoch No.:  1035
epoch No.:  1036
epoch No.:  1037
epoch No.:  1038
epoch No.:  1039
epoch No.:  1040
epoch No.:  1041
epoch No.:  1042
epoch No.:  1043
epoch No.:  1044
epoch No.:  1045
epoch No.:  1046
epoch No.:  1047
epoch No.:  1048
epoch No.:  1049
epoch No.:  1050
epoch No.:  1051
epoch No.:  1052
epoch No.:  1053
epoch No.:  1054
epoch No.:  1055
epoch No.:  1056
epoch No.:  1057
epoch No.:  1058
epoch No.:  1059
epoch No.:  1060
epoch No.:  1061
epoch No.:  1062
epoch No.:  1063
epoch No.:  1064
epoch No.:  1065
epoch No.:  1066
epoch No.:  1067
epoch No.:  1068
epoch No.:  1069
epoch No.:  1070
epoch No.:  1071
epoch No.:  1072
epoch No.:  1073
epoch No.:  1074
epoch No.:  1075
epoch No.:  1076
epoch No.:  1077
epoch No.:  1078
epoch No.:  1079
epoch No.:  1080
epoch No.:  1081
epoch No.:  1082
epoch No.:  1083
epoch No.:  1084
epoch No.:  1085
epoch No.:  10

epoch No.:  1509
epoch No.:  1510
epoch No.:  1511
epoch No.:  1512
epoch No.:  1513
epoch No.:  1514
epoch No.:  1515
epoch No.:  1516
epoch No.:  1517
epoch No.:  1518
epoch No.:  1519
epoch No.:  1520
epoch No.:  1521
epoch No.:  1522
epoch No.:  1523
epoch No.:  1524
epoch No.:  1525
epoch No.:  1526
epoch No.:  1527
epoch No.:  1528
epoch No.:  1529
epoch No.:  1530
epoch No.:  1531
epoch No.:  1532
epoch No.:  1533
epoch No.:  1534
epoch No.:  1535
epoch No.:  1536
epoch No.:  1537
epoch No.:  1538
epoch No.:  1539
epoch No.:  1540
epoch No.:  1541
epoch No.:  1542
epoch No.:  1543
epoch No.:  1544
epoch No.:  1545
epoch No.:  1546
epoch No.:  1547
epoch No.:  1548
epoch No.:  1549
epoch No.:  1550
epoch No.:  1551
epoch No.:  1552
epoch No.:  1553
epoch No.:  1554
epoch No.:  1555
epoch No.:  1556
epoch No.:  1557
epoch No.:  1558
epoch No.:  1559
epoch No.:  1560
epoch No.:  1561
epoch No.:  1562
epoch No.:  1563
epoch No.:  1564
epoch No.:  1565
epoch No.:  1566
epoch No.:  15

epoch No.:  1990
epoch No.:  1991
epoch No.:  1992
epoch No.:  1993
epoch No.:  1994
epoch No.:  1995
epoch No.:  1996
epoch No.:  1997
epoch No.:  1998
epoch No.:  1999
epoch No.:  2000
epoch No.:  2001
epoch No.:  2002
epoch No.:  2003
epoch No.:  2004
epoch No.:  2005
epoch No.:  2006
epoch No.:  2007
epoch No.:  2008
epoch No.:  2009
epoch No.:  2010
epoch No.:  2011
epoch No.:  2012
epoch No.:  2013
epoch No.:  2014
epoch No.:  2015
epoch No.:  2016
epoch No.:  2017
epoch No.:  2018
epoch No.:  2019
epoch No.:  2020
epoch No.:  2021
epoch No.:  2022
epoch No.:  2023
epoch No.:  2024
epoch No.:  2025
epoch No.:  2026
epoch No.:  2027
epoch No.:  2028
epoch No.:  2029
epoch No.:  2030
epoch No.:  2031
epoch No.:  2032
epoch No.:  2033
epoch No.:  2034
epoch No.:  2035
epoch No.:  2036
epoch No.:  2037
epoch No.:  2038
epoch No.:  2039
epoch No.:  2040
epoch No.:  2041
epoch No.:  2042
epoch No.:  2043
epoch No.:  2044
epoch No.:  2045
epoch No.:  2046
epoch No.:  2047
epoch No.:  20

...

epoch No.:  2952
epoch No.:  2953
epoch No.:  2954
epoch No.:  2955
epoch No.:  2956
epoch No.:  2957
epoch No.:  2958
epoch No.:  2959
epoch No.:  2960
epoch No.:  2961
epoch No.:  2962
epoch No.:  2963
epoch No.:  2964
epoch No.:  2965
epoch No.:  2966
epoch No.:  2967
epoch No.:  2968
epoch No.:  2969
epoch No.:  2970
epoch No.:  2971
epoch No.:  2972
epoch No.:  2973
epoch No.:  2974
epoch No.:  2975
epoch No.:  2976
epoch No.:  2977
epoch No.:  2978
epoch No.:  2979
epoch No.:  2980
epoch No.:  2981
epoch No.:  2982
epoch No.:  2983
epoch No.:  2984
epoch No.:  2985
epoch No.:  2986
epoch No.:  2987
epoch No.:  2988
epoch No.:  2989
epoch No.:  2990
epoch No.:  2991
epoch No.:  2992
epoch No.:  2993
epoch No.:  2994
epoch No.:  2995
epoch No.:  2996
epoch No.:  2997
epoch No.:  2998
epoch No.:  2999
epoch No.:  3000
epoch No.:  3001
epoch No.:  3002
epoch No.:  3003
epoch No.:  3004
epoch No.:  3005
epoch No.:  3006
epoch No.:  3007
epoch No.:  3008
epoch No.:  3009
epoch No.:  30

epoch No.:  3433
epoch No.:  3434
epoch No.:  3435
epoch No.:  3436
epoch No.:  3437
epoch No.:  3438
epoch No.:  3439
epoch No.:  3440
epoch No.:  3441
epoch No.:  3442
epoch No.:  3443
epoch No.:  3444
epoch No.:  3445
epoch No.:  3446
epoch No.:  3447
epoch No.:  3448
epoch No.:  3449
epoch No.:  3450
epoch No.:  3451
epoch No.:  3452
epoch No.:  3453
epoch No.:  3454
epoch No.:  3455
epoch No.:  3456
epoch No.:  3457
epoch No.:  3458
epoch No.:  3459
epoch No.:  3460
epoch No.:  3461
epoch No.:  3462
epoch No.:  3463
epoch No.:  3464
epoch No.:  3465
epoch No.:  3466
epoch No.:  3467
epoch No.:  3468
epoch No.:  3469
epoch No.:  3470
epoch No.:  3471
epoch No.:  3472
epoch No.:  3473
epoch No.:  3474
epoch No.:  3475
epoch No.:  3476
epoch No.:  3477
epoch No.:  3478
epoch No.:  3479
epoch No.:  3480
epoch No.:  3481
epoch No.:  3482
epoch No.:  3483
epoch No.:  3484
epoch No.:  3485
epoch No.:  3486
epoch No.:  3487
epoch No.:  3488
epoch No.:  3489
epoch No.:  3490
epoch No.:  34

epoch No.:  3914
epoch No.:  3915
epoch No.:  3916
epoch No.:  3917
epoch No.:  3918
epoch No.:  3919
epoch No.:  3920
epoch No.:  3921
epoch No.:  3922
epoch No.:  3923
epoch No.:  3924
epoch No.:  3925
epoch No.:  3926
epoch No.:  3927
epoch No.:  3928
epoch No.:  3929
epoch No.:  3930
epoch No.:  3931
epoch No.:  3932
epoch No.:  3933
epoch No.:  3934
epoch No.:  3935
epoch No.:  3936
epoch No.:  3937
epoch No.:  3938
epoch No.:  3939
epoch No.:  3940
epoch No.:  3941
epoch No.:  3942
epoch No.:  3943
epoch No.:  3944
epoch No.:  3945
epoch No.:  3946
epoch No.:  3947
epoch No.:  3948
epoch No.:  3949
epoch No.:  3950
epoch No.:  3951
epoch No.:  3952
epoch No.:  3953
epoch No.:  3954
epoch No.:  3955
epoch No.:  3956
epoch No.:  3957
epoch No.:  3958
epoch No.:  3959
epoch No.:  3960
epoch No.:  3961
epoch No.:  3962
epoch No.:  3963
epoch No.:  3964
epoch No.:  3965
epoch No.:  3966
epoch No.:  3967
epoch No.:  3968
epoch No.:  3969
epoch No.:  3970
epoch No.:  3971
epoch No.:  39

epoch No.:  4395
epoch No.:  4396
epoch No.:  4397
epoch No.:  4398
epoch No.:  4399
epoch No.:  4400
epoch No.:  4401
epoch No.:  4402
epoch No.:  4403
epoch No.:  4404
epoch No.:  4405
epoch No.:  4406
epoch No.:  4407
epoch No.:  4408
epoch No.:  4409
epoch No.:  4410
epoch No.:  4411
epoch No.:  4412
epoch No.:  4413
epoch No.:  4414
epoch No.:  4415
epoch No.:  4416
epoch No.:  4417
epoch No.:  4418
epoch No.:  4419
epoch No.:  4420
epoch No.:  4421
epoch No.:  4422
epoch No.:  4423
epoch No.:  4424
epoch No.:  4425
epoch No.:  4426
epoch No.:  4427
epoch No.:  4428
epoch No.:  4429
epoch No.:  4430
epoch No.:  4431
epoch No.:  4432
epoch No.:  4433
epoch No.:  4434
epoch No.:  4435
epoch No.:  4436
epoch No.:  4437
epoch No.:  4438
epoch No.:  4439
epoch No.:  4440
epoch No.:  4441
epoch No.:  4442
epoch No.:  4443
epoch No.:  4444
epoch No.:  4445
epoch No.:  4446
epoch No.:  4447
epoch No.:  4448
epoch No.:  4449
epoch No.:  4450
epoch No.:  4451
epoch No.:  4452
epoch No.:  44

epoch No.:  4876
epoch No.:  4877
epoch No.:  4878
epoch No.:  4879
epoch No.:  4880
epoch No.:  4881
epoch No.:  4882
epoch No.:  4883
epoch No.:  4884
epoch No.:  4885
epoch No.:  4886
epoch No.:  4887
epoch No.:  4888
epoch No.:  4889
epoch No.:  4890
epoch No.:  4891
epoch No.:  4892
epoch No.:  4893
epoch No.:  4894
epoch No.:  4895
epoch No.:  4896
epoch No.:  4897
epoch No.:  4898
epoch No.:  4899
epoch No.:  4900
epoch No.:  4901
epoch No.:  4902
epoch No.:  4903
epoch No.:  4904
epoch No.:  4905
epoch No.:  4906
epoch No.:  4907
epoch No.:  4908
epoch No.:  4909
epoch No.:  4910
epoch No.:  4911
epoch No.:  4912
epoch No.:  4913
epoch No.:  4914
epoch No.:  4915
epoch No.:  4916
epoch No.:  4917
epoch No.:  4918
epoch No.:  4919
epoch No.:  4920
epoch No.:  4921
epoch No.:  4922
epoch No.:  4923
epoch No.:  4924
epoch No.:  4925
epoch No.:  4926
epoch No.:  4927
epoch No.:  4928
epoch No.:  4929
epoch No.:  4930
epoch No.:  4931
epoch No.:  4932
epoch No.:  4933
epoch No.:  49